# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [24]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [25]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,3.80,66,98,8.45,TF,1701018237
1,1,alice springs,-23.7000,133.8833,17.75,100,100,0.00,AU,1701018237
2,2,puerto natales,-51.7236,-72.4875,9.77,51,56,8.13,CL,1701018069
3,3,stanley,54.8680,-1.6985,3.20,92,75,0.51,GB,1701017992
4,4,puerto ayora,-0.7393,-90.3518,28.98,86,57,3.13,EC,1701018237


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [26]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points( "Lng", "Lat", geo = True, tiles = "OSM", frame_width = 700,
                                        frame_height = 500, size = "Humidity", color = "City", alpha = 0.5)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [32]:
# Narrow down cities that fit criteria and drop any results with null values
sorted_city_data_df = city_data_df[(city_data_df["Max Temp"] >= 25) & (city_data_df["Max Temp"] <= 30) &
                                   (city_data_df["Humidity"] < 70) & (city_data_df["Wind Speed"] < 5)]


# Drop any rows with null values
cleaned_city_data_df = sorted_city_data_df.dropna()


# Display sample data
cleaned_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
35,35,sechura,-5.5569,-80.8222,26.09,57,4,4.49,PE,1701018240
178,178,catalao,-18.1658,-47.9464,28.99,51,91,2.41,BR,1701018255
204,204,brisas de zicatela,15.8369,-97.0419,29.29,58,0,3.09,MX,1701018258
244,244,adrar,20.5022,-10.0711,25.22,13,0,3.18,MR,1701018262
255,255,tidjikja,18.5564,-11.4271,28.87,10,0,2.02,MR,1701018263
285,285,koratla,18.8167,78.7167,25.81,63,90,2.68,IN,1701018267
332,332,atar,20.5169,-13.0499,29.35,11,0,3.54,MR,1701018246
345,345,nguigmi,14.2495,13.1092,29.54,26,84,1.27,NE,1701018272
358,358,banes,20.9625,-75.7186,29.11,67,29,4.80,CU,1701018274
376,376,copiapo,-27.3667,-70.3333,28.06,25,0,3.87,CL,1701018044


### Step 3: Create a new DataFrame called `hotel_df`.

In [37]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cleaned_city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""


# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
35,sechura,PE,-5.5569,-80.8222,57,
178,catalao,BR,-18.1658,-47.9464,51,
204,brisas de zicatela,MX,15.8369,-97.0419,58,
244,adrar,MR,20.5022,-10.0711,13,
255,tidjikja,MR,18.5564,-11.4271,10,
285,koratla,IN,18.8167,78.7167,63,
332,atar,MR,20.5169,-13.0499,11,
345,nguigmi,NE,14.2495,13.1092,26,
358,banes,CU,20.9625,-75.7186,67,
376,copiapo,CL,-27.3667,-70.3333,25,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [39]:
# Set parameters to search for a hotel
radius = 10000

categories = "accommodation.hotel"

params = {
    "categories":categories,
    "limit":1,
    "apiKey":geoapify_key    
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
sechura - nearest hotel: Maximus Inn
catalao - nearest hotel: Lindoya
brisas de zicatela - nearest hotel: Casa de Olas
adrar - nearest hotel: No hotel found
tidjikja - nearest hotel: No hotel found
koratla - nearest hotel: No hotel found
atar - nearest hotel: فندق سكليل
nguigmi - nearest hotel: Guest PAM
banes - nearest hotel: No hotel found
copiapo - nearest hotel: Hotel Diego de Almeida
dhuusamarreeb - nearest hotel: No hotel found
fonte boa - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
conceicao do araguaia - nearest hotel: No hotel found
chapada dos guimaraes - nearest hotel: Pousada Rios
delta del tigre - nearest hotel: No hotel found
vallenar - nearest hotel: Hotel del Marqués
tsaratanana - nearest hotel: No hotel found
bossangoa - nearest hotel: No hotel found
acapulco de juarez - nearest hotel: Hotel del Valle
kidal - nearest hotel: No hotel found
el galpon - nearest hotel: No hotel found
sao vicente - nearest hotel: Pousada

,City,Country,Lat,Lng,Humidity,Hotel Name
35,sechura,PE,-5.5569,-80.8222,57,Maximus Inn
178,catalao,BR,-18.1658,-47.9464,51,Lindoya
204,brisas de zicatela,MX,15.8369,-97.0419,58,Casa de Olas
244,adrar,MR,20.5022,-10.0711,13,No hotel found
255,tidjikja,MR,18.5564,-11.4271,10,No hotel found
285,koratla,IN,18.8167,78.7167,63,No hotel found
332,atar,MR,20.5169,-13.0499,11,فندق سكليل
345,nguigmi,NE,14.2495,13.1092,26,Guest PAM
358,banes,CU,20.9625,-75.7186,67,No hotel found
376,copiapo,CL,-27.3667,-70.3333,25,Hotel Diego de Almeida


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [40]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points( "Lng", "Lat", geo = True, tiles = "OSM", frame_width = 700,
                                        frame_height = 500, size = "Humidity", color = "City", alpha = 0.5,
                                   hover_cols=["Hotel Name", "Country"])


# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)